import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

In [2]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

def wndwng0d(x,snstvty=2):            #روش تشخیص ایپاک پایه
    smpl_rte=480
    i=0
    wndws=np.array([])
    #for i in range (1,2):   #print(np.shape(x))
    if snstvty==1 :                        #تشخیص ایپاک پایه
        xj=np.array(x)
        peaks,_ = find_peaks(xj, height=0.03*max(xj), distance=200,prominence=300, width=3)
        if len(peaks)<6 :
            peaks,_ = find_peaks(xj, height=0.03*max(xj), distance=160,prominence=100, width=3)
            #print('class ', i, 'Row = ', j, 'have ',len(peaks),'peaks')
        if len(peaks)>1 :
            for z in range (len(peaks)):
                if (peaks[z]<3100 and peaks[z]!=peaks[-1]) :
                    win=np.array(xj[peaks[z]:peaks[z+1]])
                    win = signal.resample(win, smpl_rte)
                    if len(win)==smpl_rte :  #win=np.append(win, i)# if len(win)==smpl_rte+1 :
                        wndws=np.append(wndws, win)


    if snstvty==2 :                        #تشخیص ایپاک حساس تر
        xj=np.array(x)
        peaks,_ = find_peaks(xj, height=0.02*max(xj), distance=160,prominence=200, width=3)
        if len(peaks)<6 :
            peaks,_ = find_peaks(xj, height=0.01*max(xj), distance=120,prominence=60, width=3)
        if len(peaks)>1 :
            for z in range (len(peaks)):
                if (peaks[z]<3100 and peaks[z]!=peaks[-1]) :
                    win=np.array(xj[peaks[z]:peaks[z+1]])
                    win = signal.resample(win, smpl_rte)
                    if len(win)==smpl_rte :
                        wndws=np.append(wndws, win)
    rows=int(len(wndws)/(smpl_rte))            #/(smpl_rte+1))
    wndws0=np.reshape(wndws,(rows,(smpl_rte)))    #(wndws,(rows,(smpl_rte+1)))
    return(wndws0)

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [3]:
cls_num=17
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)]=MITBIH(i)
os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print( i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
btch=1000

1 283	2 66	3 20	4 135	5 13	6 21	7 133	8 55	9 13	10 10	11 10	12 10	13 11	14 103	15 62	16 10	17 45	

In [4]:
np.shape(ecg)

(1000, 3601)

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    dta=np.array(vars()['ecg'+str(cls)][int(.1*len(vars()['ecg'+str(cls)])):,:]) #انتخاب محدوده ترِین جهت داده افزایی
    dta_tst=np.array(vars()['ecg'+str(cls)][:int(.1*len(vars()['ecg'+str(cls)])),:]) #انتخاب محدوده ترِین جهت داده افزایی
    vars()['wndws'+str(cls)]=wndwng(dta)                #نرخ نمونه برداری 480 است و شماره کلاس در آخر ردیف نیست
    vars()['ecg_test'+str(cls)]=wndwng(dta_tst)                #نرخ نمونه برداری 480 است و شماره کلاس در آخر ردیف نیست
    clm0vlu=np.zeros(len(vars()['wndws'+str(cls)]))
    clm0vlu_tst=np.zeros(len(vars()['ecg_test'+str(cls)]))
    cls_clm=np.array(clm0vlu)
    cls_clm_tst=np.array(clm0vlu_tst)
    cls_clm=np.int16(cls_clm+cls)
    cls_clm_tst=np.int16(cls_clm_tst+cls)
    cls_clm=np.transpose([cls_clm])                     # افزودن ستون برچسب (شماره کلاس) به سمت راست پنجره ها
    cls_clm_tst=np.transpose([cls_clm_tst])             # افزودن ستون برچسب (شماره کلاس) به سمت راست پنجره ها
    vars()['wndws'+str(cls)]=np.concatenate((vars()['wndws'+str(cls)],cls_clm),axis=1)   
    vars()['ecg_test'+str(cls)]=np.concatenate((vars()['ecg_test'+str(cls)],cls_clm_tst),axis=1)   

print("each class and its windows =")
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                         # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
cls_wndws=np.int16(cls_wndws)    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 2598	2 788	3 224	4 1564	5 158	6 189	7 1371	8 553	9 114	10 104	11 72	12 107	13 156	14 1014	15 633	16 45	17 450	
 max = 2598


In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
xtest=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    xtest=np.append(xtest,vars()['ecg_test'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))
    print('Up to class ', i, 'test shape = ', np.shape(xtest))    

Up to class  1 train shape =  (2598, 481)
Up to class  1 test shape =  (310, 481)
Up to class  2 train shape =  (3386, 481)
Up to class  2 test shape =  (377, 481)
Up to class  3 train shape =  (3610, 481)
Up to class  3 test shape =  (411, 481)
Up to class  4 train shape =  (5174, 481)
Up to class  4 test shape =  (564, 481)
Up to class  5 train shape =  (5332, 481)
Up to class  5 test shape =  (582, 481)
Up to class  6 train shape =  (5521, 481)
Up to class  6 test shape =  (602, 481)
Up to class  7 train shape =  (6892, 481)
Up to class  7 test shape =  (762, 481)
Up to class  8 train shape =  (7445, 481)
Up to class  8 test shape =  (819, 481)
Up to class  9 train shape =  (7559, 481)
Up to class  9 test shape =  (828, 481)
Up to class  10 train shape =  (7663, 481)
Up to class  10 test shape =  (837, 481)
Up to class  11 train shape =  (7735, 481)
Up to class  11 test shape =  (842, 481)
Up to class  12 train shape =  (7842, 481)
Up to class  12 test shape =  (858, 481)
Up to clas

In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[   0 2598  788  224 1564  158  189 1371  553  114  104   72  107  156
 1014  633   45  450]


In [8]:
xtrain[:,-1]

array([ 1.,  1.,  1., ..., 17., 17., 17.])

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (0,cls_num):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
xtest[:,:-1]= 2*(xtest[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtest[:,:-1]))

min trn = -1.0
max trn = 0.6348682809475628


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.array(xtest)
'''i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_test'+str(cls)],axis=0) '''  

"i=0\nwindws=np.array([])\nfor cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی\n    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)\n    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_test'+str(cls)],axis=0) "

In [12]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -1.023575967165595
max tst = 0.6348682809475628


In [13]:
len(wndws_test)

1116

In [14]:
np.shape(wndws_test)

(1116, 481)

In [15]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [16]:
fullprint(ytest)

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [17]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [18]:
np.shape(wndws1)

(2598, 481)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [19]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [20]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (10140, 480)
y_train =>  (10140,)
X_test  =>  (1116, 480)
y_test  =>  (1116,)
X_valid  =>  (10140, 480)
y_valid  =>  (10140,)


In [21]:
y_train=np.array(y_train+1)
y_test=np.array(y_test+1)
y_valid=np.array(y_valid+1)

In [22]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [23]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
127/127 [==============================] - 23s 109ms/step - loss: 1.3912 - accuracy: 0.6019 - val_loss: 2.7303 - val_accuracy: 0.1085
Epoch 2/1000
127/127 [==============================] - 13s 103ms/step - loss: 0.5259 - accuracy: 0.8183 - val_loss: 2.9549 - val_accuracy: 0.0702
Epoch 3/1000
127/127 [==============================] - 13s 105ms/step - loss: 0.3733 - accuracy: 0.8644 - val_loss: 2.1617 - val_accuracy: 0.2666
Epoch 4/1000
127/127 [==============================] - 14s 107ms/step - loss: 0.3096 - accuracy: 0.8784 - val_loss: 0.7976 - val_accuracy: 0.7307
Epoch 5/1000
127/127 [==============================] - 14s 107ms/step - loss: 0.2677 - accuracy: 0.8954 - val_loss: 0.2532 - val_accuracy: 0.8954
Epoch 6/1000
127/127 [==============================] - 14s 107ms/step - loss: 0.2512 - accuracy: 0.9014 - val_loss: 0.3538 - val_accuracy: 0.8771
Epoch 7/1000
127/127 [==============================] - 14s 109ms/step - loss: 0.2276 - accuracy: 0.9092 - val_loss: 0

KeyboardInterrupt: 

In [ ]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [ ]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

In [ ]:
        '''ecg_test=np.array(ecg1[:int(.1*len(ecg1)),:])
#print(1,len(ecg_test), end='\t')
for i in range (2,18):
    ecg_test=np.concatenate((ecg_test,vars()['ecg'+str(i)][:int(.1*len(vars()['ecg'+str(i)])),:]),axis=0)
    #print(i,len(vars()['ecg'+str(i)]), end='\t')  '''  
#print('\n ecg_test shape = ',np.shape(ecg_test))
ecg_test=np.array(xtest)
i=0
mlpinputshps=np.array([])
maxcntlbl=np.array([])
mlpinput=np.array([])
mlplbl=np.array([])
cnnlbl=np.array([])
peaklens=np.array([])            #array of peaks quantity in each raw signal
x=np.array([])
wow=0

for i in range (len(ecg_test)): #calculating max peaks in row signals
    peaklens=np.append(peaklens,len(wndwng0d(ecg_test[i])))
maxpeaks=int(np.max(peaklens))
#print(maxpeaks)

#prd_lbls_tst=np.empty((0,maxpeaks), int)              #have peaks(windows) lables for each raw signal

for i in range (len(dta_tst)):
    windws=np.array([])
    yyy=np.array([])
    yy=np.array([])
    win=np.array([])
    x=np.array(dta_tst[i,:-1])
    windws=wndwng0d(x)
    windws= 2*(windws-mn_aug)/(mx_aug - mn_aug) - 1
    windws = np.expand_dims(windws, axis=2)
    yy= model_crs.predict(windws)                    #yy= Predict each window lable for a raw signal (3600 Pnt) (17 prob in row)
    yyy=np.int16(maxindx(yy))                        # one-hot to class lable
    maxcntlbl=np.append(maxcntlbl,(np.bincount(yyy).argmax()))
    #zers=np.zeros(int(maxpeaks-len(yyy)))
    #x=np.array(np.append(yyy, zers))
    #prd_lbls_tst=np.append(prd_lbls_tst, [x], axis=0)
maxcntlbl=np.int16(maxcntlbl)
lbls=np.int16(dta_tst[:,-1])

'''cm = confusion_matrix(lbls, maxcntlbl)
print(cm)
acc3=accuracy_score(lbls, maxcntlbl)
print(classification_report(lbls, maxcntlbl))

predict_x=model_crs.predict(X_test)# Function 1
y_pred_crs=maxindx(predict_x)      # function from augment.py to remove 0 index predictions'''

rprt0=classification_report(lbls, maxcntlbl,output_dict=True)
rprt0

In [ ]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80

n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data
model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


In [ ]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

In [ ]:
        '''ecg_test=np.array(ecg1[:int(.1*len(ecg1)),:])
#print(1,len(ecg_test), end='\t')
for i in range (2,18):
    ecg_test=np.concatenate((ecg_test,vars()['ecg'+str(i)][:int(.1*len(vars()['ecg'+str(i)])),:]),axis=0)
    #print(i,len(vars()['ecg'+str(i)]), end='\t')  '''  
#print('\n ecg_test shape = ',np.shape(ecg_test))
ecg_test=np.array(xtest)
i=0
mlpinputshps=np.array([])
maxcntlbl=np.array([])
mlpinput=np.array([])
mlplbl=np.array([])
cnnlbl=np.array([])
peaklens=np.array([])            #array of peaks quantity in each raw signal
x=np.array([])
wow=0

for i in range (len(ecg_test)): #calculating max peaks in row signals
    peaklens=np.append(peaklens,len(wndwng0d(ecg_test[i])))
maxpeaks=int(np.max(peaklens))
#print(maxpeaks)

#prd_lbls_tst=np.empty((0,maxpeaks), int)              #have peaks(windows) lables for each raw signal

for i in range (len(dta_tst)):
    windws=np.array([])
    yyy=np.array([])
    yy=np.array([])
    win=np.array([])
    x=np.array(dta_tst[i,:-1])
    windws=wndwng0d(x)
    windws= 2*(windws-mn_aug)/(mx_aug - mn_aug) - 1
    windws = np.expand_dims(windws, axis=2)
    yy= model_crs.predict(windws)                    #yy= Predict each window lable for a raw signal (3600 Pnt) (17 prob in row)
    yyy=np.int16(maxindx(yy))                        # one-hot to class lable
    maxcntlbl=np.append(maxcntlbl,(np.bincount(yyy).argmax()))
    #zers=np.zeros(int(maxpeaks-len(yyy)))
    #x=np.array(np.append(yyy, zers))
    #prd_lbls_tst=np.append(prd_lbls_tst, [x], axis=0)
maxcntlbl=np.int16(maxcntlbl)
lbls=np.int16(dta_tst[:,-1])

'''cm = confusion_matrix(lbls, maxcntlbl)
print(cm)
acc3=accuracy_score(lbls, maxcntlbl)
print(classification_report(lbls, maxcntlbl))

predict_x=model_crs.predict(X_test)# Function 1
y_pred_crs=maxindx(predict_x)      # function from augment.py to remove 0 index predictions'''

rprt0=classification_report(lbls, maxcntlbl,output_dict=True)
rprt0

In [ ]:
yy

In [ ]:
yyy

In [ ]:
maxcntlbl